In [4]:
sentences = [
    "I love this product",  # Positivo
    "This is the worst experience ever",  # Negativo
    "Absolutely fantastic work",  # Positivo
    "I hate this so much",  # Negativo
    "The service was excellent",  # Positivo
    "This is a complete disaster",  # Negativo
    "I am so happy with my purchase",  # Positivo
    "This product broke after one use",  # Negativo
    "Highly recommend this to everyone",  # Positivo
    "I am deeply disappointed",  # Negativo
    "Amazing quality and performance",  # Positivo
    "The food tasted horrible",  # Negativo
    "I couldn't be more satisfied",  # Positivo
    "The delivery took forever",  # Negativo
    "The team did a great job",  # Positivo
    "The item arrived damaged",  # Negativo
    "It exceeded my expectations",  # Positivo
    "Worst customer service I've ever experienced",  # Negativo
    "The design is stunning and practical",  # Positivo
    "I regret buying this",  # Negativo
]
labels = [
    1, 0, 1, 0,  # Primeros cuatro ejemplos
    1, 0, 1, 0,  # Nuevos ejemplos
    1, 0, 1, 0,  # Más ejemplos
    1, 0, 1, 0,  # Más ejemplos
    1, 0  # Últimos dos ejemplos
]


In [3]:
from gensim.models import Word2Vec

In [5]:
tokenized_sentences = [sentence.lower().split() for sentence in sentences]

In [6]:
tokenized_sentences

[['i', 'love', 'this', 'product'],
 ['this', 'is', 'the', 'worst', 'experience', 'ever'],
 ['absolutely', 'fantastic', 'work'],
 ['i', 'hate', 'this', 'so', 'much'],
 ['the', 'service', 'was', 'excellent'],
 ['this', 'is', 'a', 'complete', 'disaster'],
 ['i', 'am', 'so', 'happy', 'with', 'my', 'purchase'],
 ['this', 'product', 'broke', 'after', 'one', 'use'],
 ['highly', 'recommend', 'this', 'to', 'everyone'],
 ['i', 'am', 'deeply', 'disappointed'],
 ['amazing', 'quality', 'and', 'performance'],
 ['the', 'food', 'tasted', 'horrible'],
 ['i', "couldn't", 'be', 'more', 'satisfied'],
 ['the', 'delivery', 'took', 'forever'],
 ['the', 'team', 'did', 'a', 'great', 'job'],
 ['the', 'item', 'arrived', 'damaged'],
 ['it', 'exceeded', 'my', 'expectations'],
 ['worst', 'customer', 'service', "i've", 'ever', 'experienced'],
 ['the', 'design', 'is', 'stunning', 'and', 'practical'],
 ['i', 'regret', 'buying', 'this']]

In [7]:
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)


In [9]:

def sentence_to_vector(sentence):
    words = sentence.lower().split()
    vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    return vectors

In [10]:
vectorized_sentences = [sentence_to_vector(sentence) for sentence in sentences]


In [17]:
len(vectorized_sentences[0][0])

100

In [12]:
len(vectorized_sentences)

20

In [22]:
min(len(seq) for seq in vectorized_sentences)

3

In [18]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Longitud máxima de las secuencias
max_sequence_length = max(len(seq) for seq in vectorized_sentences)

# Ajustamos las secuencias para que tengan la misma longitud
padded_sentences = pad_sequences(
    [np.array(seq) for seq in vectorized_sentences],
    maxlen=max_sequence_length,
    dtype='float32',
    padding='post'
)


In [21]:
len(padded_sentences[0][0])

100

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Flatten

# Creamos el modelo
model = Sequential([
    SimpleRNN(64, input_shape=(max_sequence_length, 100), activation='relu'),
    Dense(1, activation='sigmoid')  # Clasificación binaria
])

# Compilamos el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamos el modelo
model.fit(padded_sentences, np.array(labels), epochs=10, batch_size=2)
